In [ ]:
import tensorflow

import pandas as pd
import numpy as np
import os
import keras
import random
import cv2
import math
import seaborn as sns

from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split

import matplotlib.pyplot as plt

from tensorflow.keras.layers import Dense,GlobalAveragePooling2D,Convolution2D,BatchNormalization
from tensorflow.keras.layers import Flatten,MaxPooling2D,Dropout

from tensorflow.keras.applications import DenseNet201
from tensorflow.keras.applications.densenet import preprocess_input

from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator,img_to_array

from tensorflow.keras import Model

from tensorflow.keras.optimizers import Adam

from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau

import warnings
warnings.filterwarnings("ignore")

In [ ]:
# mounting data
from google.colab import drive
drive.mount('/content/drive')
path_dataset = "/content/drive/MyDrive/dataset_ML/RFMiD"
path_image_train = "/content/drive/MyDrive/dataset_ML/RFMiD/Training_Set/Training_Set/Training"
path_RFMiD_CSV = "/content/drive/MyDrive/dataset_ML/RFMiD/Training_Set/Training_Set/csvdata/RFMiD.csv"


Mounted at /content/drive


In [ ]:
model_d=DenseNet121(weights='imagenet',include_top=False, input_shape=(128, 128, 3))

x=model_d.output

x= GlobalAveragePooling2D()(x)
x= BatchNormalization()(x)
x= Dropout(0.5)(x)
x= Dense(1024,activation='relu')(x)
x= Dense(512,activation='relu')(x)
x= BatchNormalization()(x)
x= Dropout(0.5)(x)

preds=Dense(25,activation='softmax')(x) #FC-layer # 8 is the number of classification to be done
model=Model(inputs=model_d.input,outputs=preds)
model.summary()

In [ ]:
for layer in model.layers[:-8]:
    layer.trainable=False

for layer in model.layers[-8:]:
    layer.trainable=True

model.compile(optimizer='Adam',loss='categorical_crossentropy',metrics=['accuracy'])
model.summary()

In [ ]:
try:
  data =np.load("/content/drive/MyDrive/dataset_ML/RFMiD/Training_Set/Training_Set/temporary/Densenet121_data.npy")
  labels = np.load("/content/drive/MyDrive/dataset_ML/RFMiD/Training_Set/Training_Set/temporary/Densenet121_labels.npy")
except FileNotFoundError:
  data = []
  labels = []

  # Load the CSV file
  df = pd.read_csv(path_RFMiD_CSV)
  # Loop through each row in the CSV file
  for index, row in df.iterrows():
      # Load the image and resize it
      path_ = os.path.join(path_image_train+ "/" + str(row['ID'])+".png")
      print(row["ID"],end=" ")
      image = cv2.imread(path_)
      image = cv2.resize(image, (128, 128))
      print("image data added")
      image = img_to_array(image)
      data.append(image)

      # Get the labels for this image
      labels_row = [row[label] for label in df.columns if label != 'ID']
      label_array = np.array(labels_row)
      labels.append(label_array)

  # Convert the data and labels to numpy arrays
  data = np.array(data, dtype="float32") / 255.0 ##### this step is important as it converts the images pixle value from range 1-0 ######
  labels = np.array(labels)
  path_save_data="/content/drive/MyDrive/dataset_ML/RFMiD/Training_Set/Training_Set/temporary/Densenet121_data.npy"
  path_save_label="/content/drive/MyDrive/dataset_ML/RFMiD/Training_Set/Training_Set/temporary/Densenet121_labels.npy"
  np.save(path_save_data, data)
  np.save(path_save_label, labels)
else:
  print("data and labels loaded")

print(data.shape)
print(labels.shape)



In [ ]:
# train test split:
(xtrain,xtest,ytrain,ytest)=train_test_split(data,labels,test_size=0.2,random_state=42)
print(xtrain.shape, xtest.shape)
print(ytrain.shape, ytest.shape)

In [ ]:
anne = ReduceLROnPlateau(monitor='val_accuracy', factor=0.5, patience=5, verbose=1, min_lr=1e-3)
checkpoint = ModelCheckpoint('model.h5', verbose=1, save_best_only=True)

datagen = ImageDataGenerator()


datagen.fit(xtrain)
# Fits-the-model
history = model.fit_generator(datagen.flow(xtrain, ytrain, batch_size=128),
               steps_per_epoch=xtrain.shape[0] //128,
               epochs=50,
               verbose=2,
               callbacks=[anne, checkpoint],
               validation_data=(xtrain, ytrain))

In [ ]:
ypred = model.predict(xtest)

total = 0
accurate = 0
accurateindex = []
wrongindex = []

for i in range(len(ypred)):
    if np.argmax(ypred[i]) == np.argmax(ytest[i]):
        accurate += 1
        accurateindex.append(i)
    else:
        wrongindex.append(i)

    total += 1

print('Total-test-data;', total, '\taccurately-predicted-data:', accurate, '\t wrongly-predicted-data: ', total - accurate)
print('Accuracy:', round(accurate/total*100, 3), '%')

In [ ]:
label = ['DR', 'ARMD', 'MH', 'DN', 'MYA', 'BRVO',
          'TSLN', 'ERM', 'LS', 'MS', 'CSR', 'ODC',
          'CRVO', 'AH', 'ODP', 'ODE', 'AION', 'PT',
          'RT', 'RS', 'CRS', 'EDN', 'RPEC', 'MHL']
imidx = random.sample(accurateindex, k=9)# replace with 'wrongindex'

nrows = 3
ncols = 3
fig, ax = plt.subplots(nrows,ncols,sharex=True,sharey=True,figsize=(15, 12))

n = 0
for row in range(nrows):
    for col in range(ncols):
            ax[row,col].imshow(xtest[imidx[n]])
            ax[row,col].set_title("Predicted label :{}\nTrue label :{}".format(label[np.argmax(ypred[imidx[n]])], label[np.argmax(ytest[imidx[n]])]))
            n += 1

plt.show()